<a href="https://colab.research.google.com/github/Modak12/GestureControl/blob/main/2dconv%2BRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Experiment with CONV2D +RNN

In [39]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

In [40]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
train_doc = np.random.permutation(open('/content/drive/My Drive/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/My Drive/Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [43]:
print(len(train_doc))
print(len(val_doc))

663
100


In [44]:
train_doc = list(train_doc)[:50]
val_doc = list(val_doc)[:10]
batch_size = 10

In [45]:
print(len(train_doc))
print(len(val_doc))

50
10


In [46]:
dimension={
    'img_idx' : [x for x in range(30)],
    'y' : 120,
    'z' : 120
}

In [47]:
def generator(source_path, folder_list, batch_size, dimensions=dimension):

    print( 'Source path = ', source_path, '; batch size =', batch_size)

    img_idx = dimensions['img_idx']
    x = len(img_idx)
    y = dimensions['y']
    z = dimensions['z']

    while True:

        t = np.random.permutation(folder_list)
        num_batches = len(train_doc)//batch_size # calculate the number of batches
        print("Number of batches : ",num_batches)

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = resize(image,(y,z))
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255.0 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if(len(t) % batch_size):
          rem_batches = len(t) % batch_size
          print("Remaining batches : ",rem_batches)
          rem_batch_data = np.zeros((rem_batches,x,y,z,3))
          batch_labels = np.zeros((rem_batches,5))
          for folder in range(rem_batches): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*rem_batches)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*rem_batches)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = resize(image,(y,z))
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    rem_batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*rem_batches)].strip().split(';')[2])] = 1
          yield rem_batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



In [48]:
curr_dt_time = datetime.datetime.now()

train_path = '/content/drive/My Drive/Project_data/train'
val_path = '/content/drive/My Drive/Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 1 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 50
# validation sequences = 10
# epochs = 1


In [49]:
# testing generative
test_gen = generator(train_path, train_doc, 1)
d = next(test_gen)[0]

Source path =  /content/drive/My Drive/Project_data/train ; batch size = 1
Number of batches :  50


<ipython-input-47-7764a8e080dd>:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


In [50]:
np.random.seed(30)
input_shape = (30, 120, 120, 3)

In [51]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling3D, MaxPooling2D, GlobalAveragePooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.optimizers import Adam

#write your model here
sample_shape = (30, 120, 120, 3)

no_classes = 5

model = Sequential()
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(TimeDistributed(
    Conv2D(128, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())

model.add(GlobalAveragePooling3D())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))


In [52]:
optimiser = optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_12 (TimeD  (None, 30, 118, 118, 32   896       
 istributed)                 )                                   
                                                                 
 time_distributed_13 (TimeD  (None, 30, 59, 59, 32)    0         
 istributed)                                                     
                                                                 
 batch_normalization_8 (Bat  (None, 30, 59, 59, 32)    128       
 chNormalization)                                                
                                                                 
 time_distributed_14 (TimeD  (None, 30, 57, 57, 64)    18496     
 istributed)                                                     
                                                                 
 time_distributed_15 (TimeD  (None, 30, 28, 28, 64)   

In [53]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [54]:
model_name = '2d_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [55]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [56]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-56-7f8f9d39394d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
<ipython-input-47-7764a8e080dd>:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /content/drive/My Drive/Project_data/train ; batch size = 10
Number of batches :  5
5/5 [==============================] - ETA: 0s - loss: 2.0166 - categorical_accuracy: 0.2800 Source path =  /content/drive/My Drive/Project_data/val ; batch size = 10
Number of batches :  5

Epoch 1: saving model to 2d_model_init_2024-03-2206_11_05.676446/model-00001-2.01663-0.28000-1.63390-0.10000.h5
5/5 [==============================] - 79s 15s/step - loss: 2.0166 - categorical_accuracy: 0.2800 - val_loss: 1.6339 - val_categorical_accuracy: 0.1000 - lr: 0.0010
